In [27]:
from sagas.tool.servant_delegator import Delegator
from sagas.zh.hownet_helper import vis_trees

word='大学生'
# trees = hownet_procs.build_sememe_trees(word)
trees=Delegator().sememes(word=word)
trees

[{'word': {'Def': '{human|人:domain={education|教育},{study|学习:agent={~},location={InstitutePlace|场所:domain={education|教育},modifier={HighRank|高等},{study|学习:location={~}},{teach|教:location={~}}}}}',
   'en_grammar': 'noun',
   'ch_grammar': 'noun',
   'No': '041405',
   'syn': [{'id': '018725', 'text': '本科生'},
    {'id': '041583', 'text': '大专生'},
    {'id': '105566', 'text': '理工科学生'},
    {'id': '199685', 'text': '在读大学生'},
    {'id': '199686', 'text': '在读大学生'},
    {'id': '199854', 'text': '在校生'},
    {'id': '199855', 'text': '在校生'},
    {'id': '054189', 'text': '法学院学生'},
    {'id': '115413', 'text': '美院学生'},
    {'id': '189009', 'text': '医科学生'},
    {'id': '189010', 'text': '医科学生'},
    {'id': '189073', 'text': '医务助理学生'},
    {'id': '034287', 'text': '初中生'},
    {'id': '056419', 'text': '放学回家挂钥匙的学生'},
    {'id': '063723', 'text': '高中生'},
    {'id': '120511', 'text': '男生'},
    {'id': '123460', 'text': '女生'},
    {'id': '177417', 'text': '小女生'},
    {'id': '177676', 'text': '小学生'},
    {'i

In [19]:
from dataclasses import dataclass
from typing import Text, List, Dict, Set
@dataclass
class SenseSyn:
    id: str
    text: str
@dataclass        
class SenseWord:
    definition: str
    en_grammar: str
    zh_grammar: str
    en_word: str
    zh_word: str
    id: str
    syns: List[SenseSyn]

@dataclass 
class SenseNode:
    role: str
    name: str
    children: List['SenseNode']
    def role_in(self, role_name):
        return [c for c in self.children if c.role==role_name]
    @property
    def all_roles(self):
        return {c.role for c in self.children}
    
@dataclass  
class SenseTree:
    root: SenseNode
    inherits: List[SenseNode]
    roles: Dict[Text, Set[Text]]
        
data=trees[0]
word=data['word']
s_word=SenseWord(definition=word['Def'], 
                 en_grammar=word['en_grammar'],
                 zh_grammar=word['ch_grammar'],
                 en_word=word['en_word'],
                 zh_word=word['ch_word'],
                 id=word['No'],
                 syns=[SenseSyn(row['id'], row['text']) for row in word['syn']]
                )
s_word

SenseWord(definition='{human|人:domain={education|教育},{study|学习:agent={~},location={InstitutePlace|场所:domain={education|教育},modifier={HighRank|高等},{study|学习:location={~}},{teach|教:location={~}}}}}', en_grammar='noun', zh_grammar='noun', en_word='university student', zh_word='大学生', id='041405', syns=[SenseSyn(id='018725', text='本科生'), SenseSyn(id='041583', text='大专生'), SenseSyn(id='105566', text='理工科学生'), SenseSyn(id='199685', text='在读大学生'), SenseSyn(id='199686', text='在读大学生'), SenseSyn(id='199854', text='在校生'), SenseSyn(id='199855', text='在校生'), SenseSyn(id='054189', text='法学院学生'), SenseSyn(id='115413', text='美院学生'), SenseSyn(id='189009', text='医科学生'), SenseSyn(id='189010', text='医科学生'), SenseSyn(id='189073', text='医务助理学生'), SenseSyn(id='034287', text='初中生'), SenseSyn(id='056419', text='放学回家挂钥匙的学生'), SenseSyn(id='063723', text='高中生'), SenseSyn(id='120511', text='男生'), SenseSyn(id='123460', text='女生'), SenseSyn(id='177417', text='小女生'), SenseSyn(id='177676', text='小学生'), SenseSyn(id='177

In [20]:
tree_data=data['tree']
tree_data

{'role': 'sense',
 'name': '大学生',
 'children': [{'role': 'None',
   'name': 'human|人',
   'children': [{'role': 'domain', 'name': 'education|教育'},
    {'role': 'agent',
     'name': 'study|学习',
     'children': [{'role': 'location',
       'name': 'InstitutePlace|场所',
       'children': [{'role': 'domain', 'name': 'education|教育'},
        {'role': 'modifier', 'name': 'HighRank|高等'},
        {'role': 'location', 'name': 'study|学习'},
        {'role': 'location', 'name': 'teach|教'}]}]}]}]}

In [21]:
def build_node(tree_data, inherits, roles):
    role=tree_data['role']    
    node=SenseNode(role=tree_data['role'] if role!='None' else '',
                   name=tree_data['name'],
                   children=[build_node(c, inherits, roles) for c in tree_data['children']] if 'children' in tree_data else []
                  )    
    if role=='None':
        inherits.append(node)
    else:
        if role in roles:
            roles[role].add(node.name)
        else:
            roles[role]={node.name}
    return node

inherits=[]
roles={}
root=build_node(tree_data, inherits, roles)

print(root)
print([(p.name, p.all_roles, p.role_in('agent')[0].name) for p in inherits])

SenseNode(role='sense', name='大学生', children=[SenseNode(role='', name='human|人', children=[SenseNode(role='domain', name='education|教育', children=[]), SenseNode(role='agent', name='study|学习', children=[SenseNode(role='location', name='InstitutePlace|场所', children=[SenseNode(role='domain', name='education|教育', children=[]), SenseNode(role='modifier', name='HighRank|高等', children=[]), SenseNode(role='location', name='study|学习', children=[]), SenseNode(role='location', name='teach|教', children=[])])])])])
[('human|人', {'agent', 'domain'}, 'study|学习')]


In [24]:
roles

{'domain': {'education|教育'},
 'modifier': {'HighRank|高等'},
 'location': {'InstitutePlace|场所', 'study|学习', 'teach|教'},
 'agent': {'study|学习'},
 'sense': {'大学生'}}

In [22]:
s_tree=SenseTree(root=root, inherits=inherits, roles=roles)
def cat_of(tree, cat):
    rs=[ci for ci in tree.inherits if ci.name==cat]
    return len(rs)>0

cat_of(s_tree, 'human|人')

True

In [26]:
vis_trees(trees, False)
def has_role(tree:SenseTree, **kwargs):
    for k,v in kwargs.items():
        role_data=tree.roles[k]
        if v not in role_data:
            return False
    return True

has_role(s_tree, agent='study|学习')

Display #0 sememe tree
sense: 大学生
└── ✔: human|人
    ├── domain: education|教育
    └── agent: study|学习
        └── location: InstitutePlace|场所
            ├── domain: education|教育
            ├── modifier: HighRank|高等
            ├── location: study|学习
            └── location: teach|教


True

In [5]:
set([item['word']['Def'] for item in trees])

{'{human|人:domain={education|教育},{study|学习:agent={~},location={InstitutePlace|场所:domain={education|教育},modifier={HighRank|高等},{study|学习:location={~}},{teach|教:location={~}}}}}'}

In [6]:
{item['word']['Def']:item['word']['No'] for item in trees}

{'{human|人:domain={education|教育},{study|学习:agent={~},location={InstitutePlace|场所:domain={education|教育},modifier={HighRank|高等},{study|学习:location={~}},{teach|教:location={~}}}}}': '041405'}

In [7]:
vis_trees(trees[:2])

{'Def': '{human|人:domain={education|教育},{study|学习:agent={~},location={InstitutePlace|场所:domain={education|教育},modifier={HighRank|高等},{study|学习:location={~}},{teach|教:location={~}}}}}',
 'No': '041405',
 'ch_grammar': 'noun',
 'ch_word': '大学生',
 'en_grammar': 'noun',
 'en_word': 'university student',
 'syn': [{'id': '018725', 'text': '本科生'},
         {'id': '041583', 'text': '大专生'},
         {'id': '105566', 'text': '理工科学生'},
         {'id': '199685', 'text': '在读大学生'},
         {'id': '199686', 'text': '在读大学生'},
         {'id': '199854', 'text': '在校生'},
         {'id': '199855', 'text': '在校生'},
         {'id': '054189', 'text': '法学院学生'},
         {'id': '115413', 'text': '美院学生'},
         {'id': '189009', 'text': '医科学生'},
         {'id': '189010', 'text': '医科学生'},
         {'id': '189073', 'text': '医务助理学生'},
         {'id': '034287', 'text': '初中生'},
         {'id': '056419', 'text': '放学回家挂钥匙的学生'},
         {'id': '063723', 'text': '高中生'},
         {'id': '120511', 'text': '男生'},
       

In [8]:
xs={5}
xs|={6}
xs

{5, 6}